In [103]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import librosa
import numpy as np
import librosa.display
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from server.database_wrapper import PostgresqlWrapper
from server.utils import Util
from feature_extractor import FeatureExtractor
import xgboost as xgb
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Грузим датасет

In [2]:
data = list()
genre_list = list()

In [3]:
genres=['classical', 'metal', 'blues', 'hiphop', 'disco', 'pop', 'rock', 'country', 'reggae', 'jazz']

In [4]:
def getData(where_to, genre_list, genre, range_tuple):
    for i in tqdm(range(range_tuple)):
        if i < 10:
            path = "../../" + genre + "/" + genre + ".0000" + str(i) + ".au"        
        else:
            path = "../../" + genre + "/" + genre + ".000" + str(i) + ".au"
        
        song = librosa.load(path)
        where_to.append(song[0])
        genre_list.append(genre)

In [5]:
for genre in genres:
    getData(data, genre_list, genre, 100)

100%|██████████| 100/100 [01:00<00:00,  1.26it/s]


# Извлекаем фичи

In [6]:
extractor = FeatureExtractor(data)

In [7]:
low_energy = extractor.generate_energy()

Got rmse data for 0 songs
Got rmse data for 50 songs
Got rmse data for 100 songs
Got rmse data for 150 songs
Got rmse data for 200 songs
Got rmse data for 250 songs
Got rmse data for 300 songs
Got rmse data for 350 songs
Got rmse data for 400 songs
Got rmse data for 450 songs
Got rmse data for 500 songs
Got rmse data for 550 songs
Got rmse data for 600 songs
Got rmse data for 650 songs
Got rmse data for 700 songs
Got rmse data for 750 songs
Got rmse data for 800 songs
Got rmse data for 850 songs
Got rmse data for 900 songs
Got rmse data for 950 songs


In [8]:
flux = extractor.generate_flux()

Got flux data for 0 songs
Got flux data for 50 songs
Got flux data for 100 songs
Got flux data for 150 songs
Got flux data for 200 songs
Got flux data for 250 songs
Got flux data for 300 songs
Got flux data for 350 songs
Got flux data for 400 songs
Got flux data for 450 songs
Got flux data for 500 songs
Got flux data for 550 songs
Got flux data for 600 songs
Got flux data for 650 songs
Got flux data for 700 songs
Got flux data for 750 songs
Got flux data for 800 songs
Got flux data for 850 songs
Got flux data for 900 songs
Got flux data for 950 songs


In [9]:
rolloff = extractor.generate_rolloff()

Got rolloff data for 0 songs
Got rolloff data for 50 songs
Got rolloff data for 100 songs
Got rolloff data for 150 songs
Got rolloff data for 200 songs
Got rolloff data for 250 songs
Got rolloff data for 300 songs
Got rolloff data for 350 songs
Got rolloff data for 400 songs
Got rolloff data for 450 songs
Got rolloff data for 500 songs
Got rolloff data for 550 songs
Got rolloff data for 600 songs
Got rolloff data for 650 songs
Got rolloff data for 700 songs
Got rolloff data for 750 songs
Got rolloff data for 800 songs
Got rolloff data for 850 songs
Got rolloff data for 900 songs
Got rolloff data for 950 songs


In [12]:
rhythm = extractor.generate_rhythm()

Got rhythm data for 0 songs
Got rhythm data for 50 songs
Got rhythm data for 100 songs
Got rhythm data for 150 songs
Got rhythm data for 200 songs
Got rhythm data for 250 songs
Got rhythm data for 300 songs
Got rhythm data for 350 songs
Got rhythm data for 400 songs
Got rhythm data for 450 songs
Got rhythm data for 500 songs
Got rhythm data for 550 songs
Got rhythm data for 600 songs
Got rhythm data for 650 songs
Got rhythm data for 700 songs
Got rhythm data for 750 songs
Got rhythm data for 800 songs
Got rhythm data for 850 songs
Got rhythm data for 900 songs
Got rhythm data for 950 songs


In [104]:
mfcc_means = extractor.generate_mfcc(n_mfcc=20, sr=22050)

Got mfcc for 0 songs
Got mfcc for 50 songs
Got mfcc for 100 songs
Got mfcc for 150 songs
Got mfcc for 200 songs
Got mfcc for 250 songs
Got mfcc for 300 songs
Got mfcc for 350 songs
Got mfcc for 400 songs
Got mfcc for 450 songs
Got mfcc for 500 songs
Got mfcc for 550 songs
Got mfcc for 600 songs
Got mfcc for 650 songs
Got mfcc for 700 songs
Got mfcc for 750 songs
Got mfcc for 800 songs
Got mfcc for 850 songs
Got mfcc for 900 songs
Got mfcc for 950 songs


In [105]:
zcrs = extractor.generate_zero_crossing_rate(None, None)

Got zero_cross_rate for 0 songs
Got zero_cross_rate for 50 songs
Got zero_cross_rate for 100 songs
Got zero_cross_rate for 150 songs
Got zero_cross_rate for 200 songs
Got zero_cross_rate for 250 songs
Got zero_cross_rate for 300 songs
Got zero_cross_rate for 350 songs
Got zero_cross_rate for 400 songs
Got zero_cross_rate for 450 songs
Got zero_cross_rate for 500 songs
Got zero_cross_rate for 550 songs
Got zero_cross_rate for 600 songs
Got zero_cross_rate for 650 songs
Got zero_cross_rate for 700 songs
Got zero_cross_rate for 750 songs
Got zero_cross_rate for 800 songs
Got zero_cross_rate for 850 songs
Got zero_cross_rate for 900 songs
Got zero_cross_rate for 950 songs


In [21]:
cent = extractor.generate_centoid_meanstd()

Got centroid data for 0 songs
Got centroid data for 50 songs
Got centroid data for 100 songs
Got centroid data for 150 songs
Got centroid data for 200 songs
Got centroid data for 250 songs
Got centroid data for 300 songs
Got centroid data for 350 songs
Got centroid data for 400 songs
Got centroid data for 450 songs
Got centroid data for 500 songs
Got centroid data for 550 songs
Got centroid data for 600 songs
Got centroid data for 650 songs
Got centroid data for 700 songs
Got centroid data for 750 songs
Got centroid data for 800 songs
Got centroid data for 850 songs
Got centroid data for 900 songs
Got centroid data for 950 songs


In [ ]:
zcrs = extractor.generate_zero_crossing_rate(None, None)

In [22]:
# Вводим названия фичей

list_names = ["mfcc_mean_" + str(i) for i in range(1,21)]
list_names.append("std_mfcc")
list_names.append("zero_cros_mean")
list_names.append("zero_cros_std")
list_names.append("cent_mean")
list_names.append("cent_std")
list_names.append("tempo_static")
list_names.append("tempo_mean")
list_names.append("tempo_std")
list_names.append("num_tempo_changes")
list_names.append("rolloff_mean")
list_names.append("rolloff_std")
list_names.append("flux_mean")
list_names.append("flux_std")
list_names.append("energy")

# Обучаем

In [119]:
X = np.hstack((mfcc_means, zcrs, cent, rhythm, rolloff, flux, low_energy))

In [120]:
# Исключаем плохие жанры
mask = (np.array(genre_list) != 'country') & (np.array(genre_list) != 'jazz') & (np.array(genre_list) != 'reggae')

In [121]:
X = X[mask]

In [122]:
new_genre_list = np.array(genre_list)[mask]

In [131]:
# Prepare data for xgboost. Encode genres. Train/test split
le = LabelEncoder()
y = le.fit_transform(np.array(new_genre_list))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [132]:
# Normalization
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [133]:
dtrain = xgb.DMatrix(X_scaled_train, label=y_train)
dtest = xgb.DMatrix(X_scaled_test, label=y_test)

In [134]:
# Xgboost training
param = {'objective': "multi:softmax", "num_class": np.unique(y_train).size}
evallist = [(dtrain, 'train'), (dtest, 'eval')]
bst = xgb.train(param, dtrain, evals=evallist)

[0]	train-merror:0.102041	eval-merror:0.404762
[1]	train-merror:0.046939	eval-merror:0.328571
[2]	train-merror:0.034694	eval-merror:0.357143
[3]	train-merror:0.026531	eval-merror:0.357143
[4]	train-merror:0.016327	eval-merror:0.357143
[5]	train-merror:0.008163	eval-merror:0.328571
[6]	train-merror:0.004082	eval-merror:0.309524
[7]	train-merror:0.004082	eval-merror:0.290476
[8]	train-merror:0.004082	eval-merror:0.295238
[9]	train-merror:0.002041	eval-merror:0.285714


In [135]:
print(classification_report(y_test, bst.predict(dtest), target_names=['classical', 'metal', 'blues', 'hiphop', 'disco', 'pop', 'rock', 'country', 'reggae', 'jazz']))

             precision    recall  f1-score   support

  classical       0.70      0.70      0.70        30
      metal       0.90      0.93      0.92        30
      blues       0.70      0.47      0.56        30
     hiphop       0.71      0.67      0.69        30
      disco       0.70      0.87      0.78        30
        pop       0.76      0.83      0.79        30
       rock       0.52      0.53      0.52        30

avg / total       0.71      0.71      0.71       210



/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 10
  .format(len(labels), len(target_names))


In [139]:
svm = SVC(kernel='rbf', C=50)
svm.fit(X_scaled_train, y_train)

SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [140]:
print(classification_report(y_test, svm.predict(X_scaled_test), target_names=['classical', 'metal', 'blues', 'hiphop', 'disco', 'pop', 'rock', 'country', 'reggae', 'jazz']))

             precision    recall  f1-score   support

  classical       0.79      0.73      0.76        30
      metal       0.91      1.00      0.95        30
      blues       0.75      0.60      0.67        30
     hiphop       0.78      0.93      0.85        30
      disco       0.90      0.93      0.92        30
        pop       0.82      0.77      0.79        30
       rock       0.67      0.67      0.67        30

avg / total       0.80      0.80      0.80       210



/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 10
  .format(len(labels), len(target_names))


In [167]:
clf = RandomForestClassifier(n_estimators=50, )
clf.fit(X_scaled_train, y_train)
print(classification_report(y_test, clf.predict(X_scaled_test), target_names=['classical', 'metal', 'blues', 'hiphop', 'disco', 'pop', 'rock', 'country', 'reggae', 'jazz']))

             precision    recall  f1-score   support

  classical       0.74      0.57      0.64        30
      metal       0.97      0.97      0.97        30
      blues       0.50      0.43      0.46        30
     hiphop       0.78      0.60      0.68        30
      disco       0.64      0.90      0.75        30
        pop       0.73      0.80      0.76        30
       rock       0.45      0.50      0.48        30

avg / total       0.69      0.68      0.68       210



/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 10
  .format(len(labels), len(target_names))


In [ ]:
grid = GridSearchCV(svm, {"C": np.logspace(0,2), "kernel": ["poly", "rbf", "sigmoid"]}, n_jobs = -1, verbose=True)
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits


In [ ]:
param_grid = { 
    'n_estimators': [10, 20, 50, 80, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
CV_rfc.fit(X_train, y_train)

In [ ]:
param_grid = { 
    'n_estimators': [10, 20, 50, 80, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8,10,14,20,25,30,40,50],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
CV_rfc.fit(X_train, y_train)